In [1]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [2]:
# Load the dataset
mat_data = scipy.io.loadmat("B0005.mat")
battery = mat_data['B0005'][0][0]
cycles = battery['cycle'][0]

cycle_nums, capacities = [], []

for i, cycle in enumerate(cycles):
    if cycle['type'][0] == 'charge':
        d = cycle['data'][0, 0]
        current = d['Current_charge'][0]
        time = d['Time'][0].astype(float)
        delta_t = np.diff(time)
        current_avg = (current[1:] + current[:-1]) / 2
        capacity = np.sum(current_avg * delta_t) / 3600
        cycle_nums.append(i + 1)
        capacities.append(capacity)

df = pd.DataFrame({
    'Cycle': cycle_nums,
    'Capacity (Ah)': capacities
})


FileNotFoundError: [Errno 2] No such file or directory: 'B0005.mat'

In [ ]:
initial_capacity = max(capacities[:5])
df['SOH (%)'] = (df['Capacity (Ah)'] / initial_capacity) * 100


In [ ]:
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

soh_values = df['SOH (%)'].values.reshape(-1, 1)

scaler = MinMaxScaler()
soh_scaled = scaler.fit_transform(soh_values).flatten()

X, y = create_sequences(soh_scaled, seq_length=5)
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(2)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)


In [ ]:
class LSTMSOHPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, batch_first=True)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = torch.relu(self.fc1(out))
        return self.fc2(out)

model = LSTMSOHPredictor()


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

epochs = 300
for epoch in range(epochs):
    model.train()
    for xb, yb in loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    predicted = model(X_tensor).squeeze().numpy()

actual = y_tensor.squeeze().numpy()

# Denormalize
predicted_soh = predicted * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_
actual_soh = actual * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_

# RUL calculation
threshold = 80
rul = []
for i in range(len(predicted_soh)):
    rem = 0
    for j in range(i, len(predicted_soh)):
        if predicted_soh[j] >= threshold:
            rem += 1
        else:
            break
    rul.append(rem)


In [ ]:
results_df = pd.DataFrame({
    'Cycle': df['Cycle'].iloc[-len(predicted_soh):].values,
    'Capacity (Ah)': df['Capacity (Ah)'].iloc[-len(predicted_soh):].values,
    'SOH (%)': actual_soh,
    'Predicted SOH (%)': predicted_soh,
    'Predicted RUL (cycles)': rul
})
results_df.head()


In [ ]:
plt.plot(results_df['Cycle'], results_df['SOH (%)'], label='Actual')
plt.plot(results_df['Cycle'], results_df['Predicted SOH (%)'], label='Predicted')
plt.title('SOH Prediction - LSTM')
plt.xlabel('Cycle')
plt.ylabel('SOH (%)')
plt.legend()
plt.grid()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(results_df['Cycle'], results_df['Predicted RUL (cycles)'], marker='o', linestyle='-')
plt.xlabel("Cycle")
plt.ylabel("Predicted RUL (cycles)")
plt.title("📉 Predicted Remaining Useful Life (RUL) over Cycles")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(actual_soh, predicted_soh)
rmse = np.sqrt(mean_squared_error(actual_soh, predicted_soh))
r2 = r2_score(actual_soh, predicted_soh)

print(f"📊 MAE  (Mean Absolute Error): {mae:.4f}")
print(f"📊 RMSE (Root Mean Squared Error): {rmse:.4f}")

